<a id="title_ID"></a>
# JWST Pipeline Validation Testing Notebook: MIRI MRS Spec2: Assign_wcs()
<span style="color:red"> **Instruments Affected**</span>: FGS, MIRI, NIRCam, NIRISS, NIRSpec 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#JWST-Pipeline-Validation-Testing-Notebook:-MIRI-MRS" data-toc-modified-id="JWST-Pipeline-Validation-Testing-Notebook:-MIRI-MRS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>JWST Pipeline Validation Testing Notebook: MIRI MRS</a></span><ul class="toc-item"><li><span><a href="#Spec2:-Assign_wcs()" data-toc-modified-id="Spec2:-Assign_wcs()-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Spec2: Assign_wcs()</a></span><ul class="toc-item"><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Table of Contents</a></span></li></ul></li></ul></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Calibration-WG-Requested-Algorithm:" data-toc-modified-id="Calibration-WG-Requested-Algorithm:-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Calibration WG Requested Algorithm:</a></span></li></ul></li></ul></li><li><span><a href="#Loading-Data" data-toc-modified-id="Loading-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Loading Data</a></span></li><li><span><a href="#Run-JWST-Pipeline---AssignWCS" data-toc-modified-id="Run-JWST-Pipeline---AssignWCS-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Run JWST Pipeline - AssignWCS</a></span><ul class="toc-item"><li><span><a href="#Output-checks-and-tests" data-toc-modified-id="Output-checks-and-tests-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Output checks and tests</a></span></li><li><span><a href="#About-this-Notebook" data-toc-modified-id="About-this-Notebook-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>About this Notebook</a></span></li></ul></li></ul></div>

<a id="imports_ID"></a>
# Imports
List the library imports and why they are relevant to this notebook.

* os, glob for general  OS operations
* numpy
* astropy.io for opening fits files
* astropy.utils for remote data retrieval
* IPython.display for printing markdown output
* jwst.datamodels for building model for JWST Pipeline
* jwst.module.PipelineStep is the pipeline step being tested
* miricoord for coordinate conversion tools

[Top of Page](#title_ID)

In [ ]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

In [ ]:
import glob
import numpy as np
from numpy.testing import assert_allclose
import os

import astropy.io.fits as fits
import astropy.io.ascii as ascii
from ci_watson.artifactory_helpers import get_bigdata
from IPython.display import Markdown

from jwst.assign_wcs import AssignWcsStep
import miricoord.mrs.toolversions.mrs_pipetools_cdp8b as tv
from miricoord.mrs import mrs_tools as mrst

%matplotlib inline

In [ ]:
use_tempdir = True

# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
import shutil

if use_tempdir:
    data_dir = TemporaryDirectory()

    # If you have files that are in the notebook's directory, but that the notebook will need to use while
    # running, copy them into the temporary directory here.
    #
    # files = ['name_of_file']
    # for file_name in files:
    #     shutil.copy(file_name, os.path.join(data_dir.name, file_name))

    # Save original directory
    orig_dir = os.getcwd()

    # Move to new directory
    os.chdir(data_dir.name)

# For info, print out where the script is running
print("Running in {}".format(os.getcwd()))


<a id="intro_ID"></a>
# Introduction


In this notebook we will test the assign_wcs step of Spec2Pipeline() for **MRS** observations.

Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/index.html#assign-wcs-step

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/assign_wcs


### Calibration WG Requested Algorithm: 

https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Spectral+GWCS+Information



[Top of Page](#title_ID)

<a id="data_ID"></a>
# Loading Data

We are using simulated MRS observations, generated with MIRISim v2.3.0 (as of Dec 2020). ***(Needs updating)*** 

[Top of Page](#title_ID)

In [ ]:
mock12A = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock12A-0.fits')

mock12B = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock12B-0.fits')

mock12C = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock12C-0.fits')

mock34A = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock34A-0.fits')

mock34B = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock34B-0.fits')

mock34C = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock34C-0.fits')

files = [mock12A, mock12B, mock12C, mock34A, mock34B, mock34C]

Collect configuration files locally if they aren't yet there.

In [ ]:
# if not os.path.exists('../cfg_files/'):
#     os.mkdir('../cfg_files/')
#     cfgs = collect_pipeline_cfgs.collect_pipeline_cfgs(dst='../cfg_files/')

<a id="pipeline_ID"></a>
# Run JWST Pipeline - AssignWCS

For this test we do not need to run the data through the Detector1() pipeline (in fact, this data will fail in Detector1). We begin by running the data through AssignWCS in spec2 using the 'assign_wcs.cfg' file. The AssignWcs step should not change anything in the science data or error extension. It attaches the WCS information to the data, so that a location in detector pixels can be reconstructed to an RA, dec, wavelength in spatial and spectral space.

[Top of Page](#title_ID)

In [ ]:
awcs_out = []

for dd in files: #det1_out:
    awcs = AssignWcsStep.call(dd, save_results=True)
    awcs_out.append(awcs)

<a id="residual_ID"></a>
## Output checks and tests

We can use the output model to run the following checks:

* the SCI extension is unchanged from the step input (accept default tolerances for the np.allclose() function)
* we use the wcs information assigned in the step to check the RA, dec, wavelength assignment at some reference locations. 


[Top of Page](#title_ID)

In [ ]:
# Do a numeric check between input and output
for (aa, dd) in zip(awcs_out, files):
    data = fits.getdata(dd)
    filename = os.path.split(dd)[-1]
    assert np.allclose(aa.data, data, equal_nan=True), "Step input and output don't match for {}".format(filename)

Load the set of test coordinates from ``miricoord``. Then calculate the wavelength of these coordinates from the assign_wcs output models, and verify that they are close. If all the ``assert`` tests pass, this test is successful and complete.

In [ ]:
def verify_coords(imgnum, ch, ref_data):
    """Uses predefined test cooridnates to check WCS output."""
    
    xt = ref_data[ch]['x']
    yt = ref_data[ch]['y']
    v2t = ref_data[ch]['v2']
    v3t = ref_data[ch]['v3']
    lamt = ref_data[ch]['lam']
    
    lamtest1 = np.zeros(2)
    lamtest2 = np.zeros(2)
    atest1 = np.zeros(2)
    btest1 = np.zeros(2)
    v2test1 = np.zeros(2)
    v3test1 = np.zeros(2)
    lt1 = np.zeros(2)
    
    print('**** TEST SET {}****'.format(ch))
    print(' x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test')
    for i in range (2):
        values = mrst.xytoabl(xt[i], yt[i], ch)
        atest1[i], btest1[i], lt1[i] = values['alpha'], values['beta'], values['lam']
        v2test1[i], v3test1[i] = mrst.abtov2v3(atest1[i], btest1[i], ch)
        ra, dec, lamtest1[i] = awcs_out[imgnum].meta.wcs(xt[i], yt[i])
        print('{0}  {1}   {2:.5f}    {3:.5f}    {4:.5f}   {5:.5f}   {6:.5f}   {7:.5f}'.format(
            xt[i], yt[i], lamt[i], lamtest1[i], v2t[i], v3t[i], v2test1[i], v3test1[i]))

        assert_allclose(lamt[i], lamtest1[i], atol=0.05)
        assert_allclose(v2t[i], v2test1[i], atol=0.05)
        assert_allclose(v3t[i], v3test1[i], atol=0.05)
        
    print('\n')

In [ ]:
mrd = tv.mrs_ref_data
chan = {'0':'1A',
        '1':'1B',
        '2':'1C',
        '3':'2A',
        '4':'2B',
        '5':'2C',
        '6':'3A',
        '7':'3B',
        '8':'3C',
        '9':'4A',
        '10':'4B',
        '11':'4C'}

for i in range(12):
    ch = chan[str(i)]
    
    if i < 6 and 'A' in ch:
        imgnum = 0
    elif i < 6 and 'B' in ch:
        imgnum = 1
    elif i < 6 and 'C' in ch:
        imgnum = 2
    elif i >= 6 and 'A' in ch:
        imgnum = 3
    elif i >= 6 and 'B' in ch:
        imgnum = 4
    elif i >= 6 and 'C' in ch:
        imgnum = 5
        
    verify_coords(imgnum, ch, mrd)

<a id="about_ID"></a>
## About this Notebook
**Author:** Mike Engesser, Science Support Analyst, MIRI branch 
<br>(Adapted from Sarah Kendrew's AssignWCS LRS-SLIT notebook.)
<br>**Updated On:** 4/01/2021

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 